In [2]:
# Setting up three graph theory
K4 = ThreeGraphTheory(4, edges=[[0, 1, 2], [1, 2, 3], [0, 2, 3], [0, 1, 3]])
C5 = ThreeGraphTheory.p(5, edges=[[0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 0], [4, 0, 1]])
ThreeGraphTheory.exclude([K4, C5])
TGp = ThreeGraphTheory

# Setting up colored theory
TwoColorTheory = combine("2Colors", Color0, Color1, symmetries=NoSymmetry)
TwoColorTheory.exclude([TwoColorTheory(1), TwoColorTheory(1, C0=[0], C1=[0])])
CTG = combine("2ColorThreeGraphs", TGp, TwoColorTheory)

# Testing the generation of structures
print("Number of structures without C5, K4")
print("and size 5: ", len(TGp.generate(5)), " (should be 20)")
print("and size 6: ", len(TGp.generate(6)), " (should be 572)")

print("\nNumber of 2 colored structures without C5, K4")
print("and size 5: ", len(CTG.generate(5)), " (should be 338)")
print("and size 6: ", len(CTG.generate(6)), " (should be 28080)")


target_size = 6

# Edge with correct colors
C = CTG(3, edges=[[0, 1, 2]], C0=[0, 1], C1=[2])
# Missing edge with good colors
M = CTG(3, edges=[], C0=[0, 1], C1=[2])
# Pointed edge with correct colors, point from color 0
Cp0 = CTG(3, edges=[[0, 1, 2]], C0=[0, 1], C1=[2], ftype=[0])
# Pointed edge with correct colors, point from color 1
Cp1 = CTG(3, edges=[[0, 1, 2]], C0=[0, 1], C1=[2], ftype=[2])


# Edge with bad colors all in color 0
B000 = CTG(3, edges=[[0, 1, 2]], C0=[0, 1, 2], C1=[])
# Edge with bad colors, looking the wrong way
B011 = CTG(3, edges=[[0, 1, 2]], C0=[0], C1=[1, 2])
# Pointed edge with bad colors, point from moving 1 -> 0
Bp0 = CTG(3, edges=[[0, 1, 2]], C0=[0, 1, 2], C1=[], ftype=[0])
# Pointed edge with bad colors, point from moving 0 -> 1
Bp1 = CTG(3, edges=[[0, 1, 2]], C0=[0], C1=[1, 2], ftype=[2])

# Degree equality conditions for various type pairs
degeq_00 = CTG.p(4, ftype=[0, 1], edges=[[0, 2, 3]], edges_m=[[1, 2, 3]], C0=[0, 1]) - \
           CTG.p(4, ftype=[1, 0], edges=[[0, 2, 3]], edges_m=[[1, 2, 3]], C0=[0, 1])
degeq_01 = CTG.p(4, ftype=[0, 1], edges=[[0, 2, 3]], edges_m=[[1, 2, 3]], C0=[0], C1=[1]) - \
           CTG.p(4, ftype=[0, 1], edges=[[1, 2, 3]], edges_m=[[0, 2, 3]], C0=[0], C1=[1])
degeq_11 = CTG.p(4, ftype=[0, 1], edges=[[0, 2, 3]], edges_m=[[1, 2, 3]], C1=[0, 1]) - \
           CTG.p(4, ftype=[1, 0], edges=[[0, 2, 3]], edges_m=[[1, 2, 3]], C1=[0, 1])

# Assumptions used in the proof
assums = [Cp0 - Bp0, Cp1 - Bp1, degeq_00, -degeq_00, degeq_01, -degeq_01, degeq_11, -degeq_11, C - 34/100]

# Close rational approximation for gamma: 2521/5432
gamma = continued_fraction(2*sqrt(3) - 3).convergent(7)
# Optimal construction and its derivatives
symbolic_constr = CTG.blowup_construction(target_size, 2, edges=[[0, 0, 1]], C0=[[0]], C1=[[1]], symbolic_parts=True).set_sum()
ders = symbolic_constr.derivatives([gamma])

# Only use a subset of the types
all_types = CTG._get_relevant_ftypes(target_size)
idx_good_types = [7, 6, 5, 4, 3, 15, 14, 13, 12, 11, 10, 9, 8, 24, 23, 22, 21, 20, 19]
ftype_subset = [all_types[ii] for ii in idx_good_types]

Number of structures without C5, K4
and size 5:  20  (should be 20)
and size 6:  572  (should be 572)

Number of 2 colored structures without C5, K4
and size 5:  338  (should be 338)
and size 6:  28080  (should be 28080)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 12.61it/s]


In [3]:
CTG.solve_sdp(B000 + B011 - M, target_size, maximize=True,
              specific_ftype=ftype_subset, positives=assums,
              construction=ders, file="certificates/c5_k4_sdp", 
              sdp_params={
                  "objtol":"1.0e-9", 
                  "maxiter":"300"
              })

Base flags generated, their number is 28080
The relevant ftypes are constructed, their number is 19
Block sizes before symmetric/asymmetric change is applied: [128, 128, 128, 128, 128, 112, 112, 112, 112, 112, 112, 112, 112, 82, 82, 82, 82, 82, 82]


Done with mult table for Ftype on 4 points with edges=(012 023), C0=(2 3), C1=(0 1): : 19it [00:09,  2.08it/s] 


Adjusting table with kernels from construction
Tables finished


Done with positivity constraint 8: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [08:19<00:00, 55.45s/it]


Constraints finished
Running SDP. Used block sizes are [21, 106, 40, 88, 56, 72, 39, 88, 21, 106, 32, 80, 70, 42, 32, 80, 68, 44, 68, 44, 32, 80, 68, 44, 32, 80, 39, 43, 50, 32, 55, 27, 35, 46, 36, 46, 82, -28080, -304]
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 1.03e-02 Pobj: -2.2765741e+00 Ad: 6.93e-02 Dobj:  1.9133058e+03 
Iter:  2 Ap: 1.29e-01 Pobj: -2.3879035e+01 Ad: 5.41e-02 Dobj:  2.6166194e+03 
Iter:  3 Ap: 2.70e-01 Pobj: -6.0302109e+01 Ad: 2.65e-01 Dobj:  3.0370900e+03 
Iter:  4 Ap: 9.55e-01 Pobj: -1.7239710e+02 Ad: 3.73e-01 Dobj:  2.3268937e+03 
Iter:  5 Ap: 1.00e+00 Pobj: -1.9459670e+02 Ad: 7.91e-01 Dobj:  5.4840156e+02 
Iter:  6 Ap: 1.00e+00 Pobj: -2.0574913e+02 Ad: 8.91e-01 Dobj:  6.4421769e+01 
Iter:  7 Ap: 1.00e+00 Pobj: -2.1691278e+02 Ad: 9.44e-01 Dobj:  3.7870583e+00 
Iter:  8 Ap: 1.00e+00 Pobj: -2.4222538e+02 Ad: 8.83e-01 Dobj:  4.9067938e-01 
Iter:  9 Ap: 1.00e+00 Pobj: -3.2556196e+02 Ad: 6.10e-01 Dobj:  2.16

-1.0239652788873732e-10

In [4]:
CTG.round_solution("certificates/c5_k4_sdp", certificate_file="certificates/c5_k4_cert", 
                   round_params={
                       "denom": 1024 * 16 * 3 * 5, 
                       "slack_threshold": 1e-6
                   })

Starting the rounding of the result
Flattening X matrices
This took 64.44038009643555s
Correcting flat X matrices
Dimensions:  (612, 58245)
This took 40.80783653259277s
Unflattening X matrices
This took 0.0006084442138671875s
Calculating resulting bound


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [1:08:28<00:00, 216.24s/it]


This took 4111.464602947235s
Final rounded bound is 0


0

In [ ]:
K4 = ThreeGraphTheory(4, edges=[[0, 1, 2], [1, 2, 3], [0, 2, 3], [0, 1, 3]])
C5 = ThreeGraphTheory.pattern(5, edges=[[0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 0], [4, 0, 1]])
ThreeGraphTheory.exclude([K4, C5])
TGp = ThreeGraphTheory
TwoColorTheory = combine("2Colors", Color0, Color1, symmetries=NoSymmetry)
TwoColorTheory.exclude([TwoColorTheory(1), TwoColorTheory(1, C0=[0], C1=[0])])
CTG = combine("2ColorThreeGraphs", TGp, TwoColorTheory)

C = CTG(3, edges=[[0, 1, 2]], C0=[0, 1], C1=[2])
M = CTG(3, edges=[], C0=[0, 1], C1=[2])
Cp0 = CTG(3, edges=[[0, 1, 2]], C0=[0, 1], C1=[2], ftype=[0])
Cp1 = CTG(3, edges=[[0, 1, 2]], C0=[0, 1], C1=[2], ftype=[2])
B000 = CTG(3, edges=[[0, 1, 2]], C0=[0, 1, 2], C1=[])
B011 = CTG(3, edges=[[0, 1, 2]], C0=[0], C1=[1, 2])
Bp0 = CTG(3, edges=[[0, 1, 2]], C0=[0, 1, 2], C1=[], ftype=[0])
Bp1 = CTG(3, edges=[[0, 1, 2]], C0=[0], C1=[1, 2], ftype=[2])
degeq_00 = CTG.p(4, ftype=[0, 1], edges=[[0, 2, 3]], edges_m=[[1, 2, 3]], C0=[0, 1]) - \
           CTG.p(4, ftype=[1, 0], edges=[[0, 2, 3]], edges_m=[[1, 2, 3]], C0=[0, 1])
degeq_01 = CTG.p(4, ftype=[0, 1], edges=[[0, 2, 3]], edges_m=[[1, 2, 3]], C0=[0], C1=[1]) - \
           CTG.p(4, ftype=[0, 1], edges=[[1, 2, 3]], edges_m=[[0, 2, 3]], C0=[0], C1=[1])
degeq_11 = CTG.p(4, ftype=[0, 1], edges=[[0, 2, 3]], edges_m=[[1, 2, 3]], C1=[0, 1]) - \
           CTG.p(4, ftype=[1, 0], edges=[[0, 2, 3]], edges_m=[[1, 2, 3]], C1=[0, 1])

assums = [Cp0 - Bp0, Cp1 - Bp1, degeq_00, -degeq_00, degeq_01, -degeq_01, degeq_11, -degeq_11, C - 34/100]

CTG.verify("certificates/c5_k4_cert", B000 + B011 - M, 6, positives=assums)

Checking X matrices


19it [00:17,  1.07it/s]


Solution matrices are all positive semidefinite, linear coefficients are all non-negative
Calculating multiplication tables


19it [00:07,  2.66it/s]
